In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize
import scipy.integrate

# Ordinary Differential Equation Methods

There are many possible types of ODE and PDE you may want to try to solve numerically, and many different approaches to try to do this, each with their own advantages and disadvantages.

Let's begin with some methods for solving first order ODEs.

## (Forward) Euler Method

This is the simplest way to solve an ordinary differential equation. While there are much better methods to use in practice, it should give you a good idea of what we're trying to do.

Say we're given an initial value for the position of an atom, and we have an an expression for its velocity:
$$ x(t=0) = c; \qquad \dot{x}(t) = f(t, x(t)). $$

From this we want to calculate the position of the atom as a function of time. 
- We first calculate its velocity at its initial position.
- Then we use this to estimate where the atom will be some timestep $h$ later.
- This gives us a new postion, at which we calculate a new velocity and repeat this process to caclulate a full trajectory.

So we have an iterative procedure we use to calculate $x_n$ which is our approximation for the value of $x$ at time $t_n = t_0 + n h$:
- $x_0 = c$
- $x_{n+1} = x_n + f(t_n, x_n)h $

You can probably guess that the accuracy of this will depend strongly on how small a timestep we use, and we will potentially get further and further from the real solution as we progress.

Let's write a python function to calculate this

In [ ]:
def euler(x0, xprime, h, tmax):
    '''Euler method'''
    
    nt = int(tmax/h)
    xt = np.empty(nt+1)
    tvals = np.linspace(0, nt*h, nt+1)
    xt[0] = x0
    for it in range(nt):
        xt[it + 1] = xt[it] + h*xprime(tvals[it], xt[it])
    return tvals, xt

In [ ]:
# Let's try a simple function for our velocity
def v1(t, x):
    return x*np.cos(t)

# And compare the result for different step sizes.
t1, x1 = euler(1, v1, 0.5, 20)
t2, x2 = euler(1, v1, 0.2, 20)
t3, x3 = euler(1, v1, 0.1, 20)
t4, x4 = euler(1, v1, 0.001, 20)
plt.plot(t1, x1, label="dt=0.4")
plt.plot(t2, x2, label="dt=0.2")
plt.plot(t3, x3, label="dt=0.1")
plt.plot(t4, x4, label="dt=0.001")
plt.legend()
plt.show()

### Local Truncation Error

The local truncation error is the error made in a single step of the method, i.e. the difference between the numerical solution after 1 step $x_1$, and the exact solution $x(t_1)$.

If we write $x_1 = x_0 + f(x_0, t_0) h$, and compare this to the Taylor expansion of $x(t)$ around $t=t_0$: $x(t_0+h)=x(t_0) + h x'(t_0)+\frac{h^2}{2}x''(t_0) + O(h^3) $, we can estimate the local truncation error as

$$ x(t_0+h) - x_1 = \frac{1}{2} h^2 x''(t_0) + O(h^3). $$

This shows that for small $h$, the local truncation error is proportional to $h^2$.

### Global Truncation Error

The global truncation error is the error incurred by the method at a given time $t$, and is the total of all the local trunctation errors made at each step to get to that time.

The number of steps to reach time $t$ is given by $\frac{t-t_0}{h}$. This is proportional to $\frac{1}{h}$. With the local trunctaion error proportional to $h^2$ at each step, the global truncation error will be proportional to $h$.

As the global truncation error is proportional to $h$, the Euler method is termed a _first order_ method.

### Stability

Stability can be an issue for the Euler method, meaning that our approximate solution can diverge rapidly, or oscillate with increasing amplitude around the true solution in certain cases.

In [ ]:
# The Euler method can be unstable.
# For example:
def v2(t, x):
    return -4*x

t1, x1 = euler(1, v2, 0.6, 5)
t2, x2 = euler(1, v2, 0.5, 5)
t3, x3 = euler(1, v2, 0.2, 5)
t4, x4 = euler(1, v2, 0.001, 5)
plt.plot(t1, x1, label="dt=1.0")
plt.plot(t2, x2, label="dt=0.5")
plt.plot(t3, x3, label="dt=0.2")
plt.plot(t4, x4, label="dt=0.001")
plt.legend()
plt.show()

## Backward Euler Method

The backward Euler method is a small modification of the forward Euler method given by

$$ x_{n+1} = x_n + f(t_{n_1+1}, x_{n+1})h$$

This will eliminate the stability problem mentioned above, but our equation is now implicit. This method is sometimes referred to as the implicit Euler method for this reason. We'll need to solve an equation for $x_{n+1}$ at each timestep, which will make this approach much more time consuming.

This could be done, for example with one of the root finding methods covered before to solve $ x_{n+1} - x_n - f(t_{n_1+1}, x_{n+1})h= 0 $ for $x_{n+1}$.

Let's write a simple Python implementation, using Newton's method.

In [ ]:
def x_newton(xi, xprime, t, h):
    '''find x_i+1 for backward Euler'''
    # Define a function we can pass to a root finding method
    def root_func(x):
        return x - xi - xprime(t, x) * h
    return scipy.optimize.newton(root_func, xi)

def back_euler(x0, xprime, h, tmax):
    '''Backward Euler method'''
    nt = int(tmax/h)
    xt = np.empty(nt+1)
    tvals = np.linspace(0, nt*h, nt+1)
    xt[0] = x0
    for it in range(nt):
        xt[it+1] = x_newton(xt[it], xprime, tvals[it], h)
    return tvals, xt

In [ ]:
# Let's see if this solves the stability issue we had before:
# We can use an even bigger timestep here.
t1, x1 = back_euler(1, v2, 1.0, 5)
t2, x2 = back_euler(1, v2, 0.5, 5)
t3, x3 = back_euler(1, v2, 0.2, 5)
t4, x4 = back_euler(1, v2, 0.001, 5)
plt.plot(t1, x1, label="dt=1.0")
plt.plot(t2, x2, label="dt=0.5")
plt.plot(t3, x3, label="dt=0.2")
plt.plot(t4, x4, label="dt=0.001")
plt.legend()
plt.show()

In [ ]:
# However, not everything is better...
t3, x3 = back_euler(1, v1, 0.2, 20)
t4, x4 = back_euler(1, v1, 0.001, 20)

plt.plot(t3, x3, label="dt=0.2")
plt.plot(t4, x4, label="dt=0.001")
plt.legend()
plt.show()

## Runge-Kutta Methods

Runge-Kutta methods are a general class of methods for solving the ODE $\frac{dx}{dt} = f(t, x)$. We'll start by looking at a **second order Runge-Kutta method**.

First let's again look the Taylor expansion of $x(t)$: $x(t+h)=x(t) + h x'(t)+\frac{h^2}{2}x''(t) + O(h^3) $. Another way we could write this is $x(t+h)=x(t) + h x'(t)+\frac{h^2}{2}x''(\xi)$, where $\xi$ is some value between $t$ and $t+h$ that makes this expression exact. We can use this to express the first derivative as
$$
x'(t) = \frac{x(t+h)-x(t)}{h} - \frac{h}{2}x''(\xi).
$$

Now we can write
$$
x(t+h) = x(t) + hx'(\xi) = x(t) + hf(\xi, x(\xi))
$$
which will be exact for some value of $\xi$ between $t$ and $t+h$. The Euler method uses $\xi=t$, while the backward Euler method uses $\xi=t+h$. Instead, we can use $\xi=t+h/2$ as this is likely a better guess.

However, we don't know the value $x(t+h/2)$. What we can do instead is estimate it using Euler's method.

Let's implement this in a Python function:

In [ ]:
def RK2(x0, xprime, h, tmax):
    '''Second order Runge-Kutta method'''

    nt = int(tmax/h)
    xt = np.empty(nt+1)
    tvals = np.linspace(0, nt*h, nt+1)
    xt[0] = x0
    for it in range(nt):
        x_halfstep = xt[it] + 0.5*h*xprime(tvals[it], xt[it])
        xt[it + 1] = xt[it] + h*xprime(tvals[it] + 0.5*h, x_halfstep)
    return tvals, xt

In [ ]:
t1, x1 = RK2(1, v1, 1.0, 20)
t2, x2 = RK2(1, v1, 0.001, 20)
t3, x3 = euler(1, v1, 0.5, 20)
plt.plot(t1, x1, label="RK2 dt=1.0")
plt.plot(t2, x2, label="RK2 dt=0.001")
plt.plot(t3, x3, label="Euler dt=0.5")
plt.legend()
plt.show()

As you can see, this performs much better than the Euler method, even if we half the Euler timestep, so that we have the same number of function evaluations.

The second order Runge-Kutta given here, is just one of many possible variants. We can write the general form of the second order Runge-Kutta as

$$
x(t+h) = x(t) + w_1 h f(x(t), t) + w_2 h f(t+\alpha h, x^*),
$$

with $x^*(t+h) = x(t) + \beta h f(t, x(t)).$
For the second-order method we implemented we had $w_1=0$, $w_2=1$, $\alpha=1/2$, and $\beta=1/2$. Another equally valid method would have  $w_1=1/2$, $w_2=1/2$, $\alpha=1$, and $\beta=1$, which could be obtained from a two variable Taylor expansion of $f(t, x)$.

As you may have guessed, these are second-order methods: the global truncation error will be $O(h^2)$.

## Fourth-Order Runge-Kutta

The method usually referred to as _the_ Runge-Kutta method, or the classical Runge-Kutta method, is the fourth-order method. This is likely the most common method used for solving this type of differential equation due to its balance of accuracy and efficiency.

In the fourth-order method we write

$$
x(t+h) = x(t) + \frac{h}{6}(k_1+2k_2+2k_3+k_4)
$$

where

$$
k_1 = f(t, x)\\
k_2 = f(t+\frac{h}{2}, x+\frac{h}{2}k_1)\\
k_3 = f(t+\frac{h}{2}, x+\frac{h}{2}k_2)\\
k_4 = f(t+h, x+hk_3)\\
$$

Let's write a Python implementation as before:

In [ ]:
def RK4(x0, xprime, h, tmax):
    '''Fourth order Runge-Kutta method'''
    
    nt = int(tmax/h)
    xt = np.empty(nt+1)
    tvals = np.linspace(0, nt*h, nt+1)
    xt[0] = x0
    for it in range(nt):
        
        k1 = xprime(tvals[it], xt[it])
        k2 = xprime(tvals[it] + 0.5*h, xt[it] + 0.5*h*k1)
        k3 = xprime(tvals[it] + 0.5*h, xt[it] + 0.5*h*k2)
        k4 = xprime(tvals[it] + h, xt[it] + h*k3)
        
        xt[it + 1] = xt[it] + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    return tvals, xt

In [ ]:
t1, x1 = RK4(1, v1, 1.0, 20)
t2, x2 = RK4(1, v1, 0.001, 20)
t3, x3 = RK2(1, v1, 1.0, 20)
plt.plot(t1, x1, "ro", label="RK4 dt=1.0")
plt.plot(t2, x2, "b--", label="RK4 dt=0.001")
plt.plot(t3, x3, "g+", label="RK2 dt=1.0")
plt.legend()
plt.show()

## Implicit RK

As with the backward Euler, there many implicit Runge-Kutta methods also exist. In the explict methods discussed so far the value of the solution at the current step is determined solely by the information from the previous steps. As with the forward Euler method, this can lead to stability issues unless the step size is small enough for certain types of systems. The implicit methods alleviate this, as we saw with the backward Euler method. In the implicit methods, a system of equations must be solved at each step, so these tend to be somewhat slower.

## Adaptive Step Size

In the same way as we could use adaptive interval sizes in numerical integration, we can use an adaptive step size when numerically solving a differential equation. This will allow us to automatically take a smaller stepsize in regions where the solution is varying more rapidly, and take a larger stepsize where it varies more smoothly.

Runge-Kutta methods can enable this to be done quite efficiently by combining a method of order $p$ with a method of order $p-1$ to provide an estimate of the error at each step, as the difference between the value obtained with the two methods.

## In SciPy

The [scipy.integrate](https://docs.scipy.org/doc/scipy/reference/integrate.html) module has solvers for initial value ODE problems, with the main function that can be used called [scipy.integrate.solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html). You may need to update your SciPy version if you get an error when you try to use this function. This allows several methods to be chosen between, with the default RK45. This RK45 method actually implements the [Dormand-Prince](https://en.wikipedia.org/wiki/Dormand%E2%80%93Prince_method) RK method. This calculates both fourth and fifth order solutions and uses the difference between them to provide an error estimate the error of the fourth order solution, which can be used for an adaptive step size.

The methods implemented in `solve_ivp` can also be called directly as e.g. [scipy.integrate.RK45](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.RK45.html).

If you have a boundary value problem, rather than an initial value problem, while we haven't discussed them here, you can use the [scipy.integrate.solve_bvp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_bvp.html) function.

If you have an older SciPy version, there is the [scipy.integrate.odeint](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html) function available. This calls the "lsoda" method from ODEPACK. This may be deprecated in the future, so I suggest using the newer version if available.

Let's try our example problem with `solve_ivp`:

In [ ]:
sp_res = scipy.integrate.solve_ivp(fun=v1, t_span=(0, 20), y0=[1], method='RK45')
print(sp_res)

In [ ]:
plt.plot(sp_res.t, sp_res.y[0], "ro", label="SciPy RK45")
plt.plot(t2, x2, "b--", label="Our RK4 dt=0.001")
plt.legend()
plt.show()

In [ ]:
# Say we want to find the values at specific times, we can
# pass an array of t -values to the t_eval argument.
tvalues = np.linspace(0, 20, 100)

sp_res2 = scipy.integrate.solve_ivp(fun=v1, t_span=(0, 20),
                                    y0=[1], method='RK45',
                                    t_eval=tvalues)
plt.plot(sp_res.t, sp_res.y[0], "ro", label="Auto Points")
plt.plot(tvalues, sp_res2.y[0], "b--", label="Chosen Points")
plt.legend()
plt.show()

In [ ]:
# Or if we want some general dense output, there's
# a flag to turn this on, that generates an object
# in .sol that will return values for any input
# in the calculated range.
sp_res3 = scipy.integrate.solve_ivp(fun=v1, t_span=(0, 20),
                                    y0=[1], method='RK45',
                                    dense_output=True)

# Now we can use sp_res3.sol to produce a value at any t-value
plt.plot(sp_res.t, sp_res.y[0], "ro", label="Auto Points")
plt.plot(tvalues, sp_res3.sol(tvalues)[0], "b--", label="Dense Points")
plt.legend()
plt.show()

The above mentioned SciPy functions are mostly Python implementations, except the "lsoda" method which is a wrapper to the ODEPACK Fortran library.

## Second order ODEs

To solve a second order differential equation, one approach is to convert it to a set of first order differential equations. Lets try the equation of a damped simple harmonic oscillator:

$$
\frac{\mathrm d^2 x}{\mathrm d t^2} + \gamma \frac{\mathrm d x}{\mathrm d t} + \omega_0 x = 0.
$$

If we denote $v = \dot x$ we can write this as the pair of first order equations:

$$
v = \frac{\mathrm d x}{\mathrm d t} \\
\frac{\mathrm d v}{\mathrm d t} = - \gamma v - \omega_0 x.
$$

We can solve this with the SciPy ODE solver by constructing a function that takes the array $y=(x, v)$ as an argument and returns the array $\mathrm d y/\mathrm d t= (\mathrm dx/\mathrm dt, \mathrm dv/\mathrm dt)$ as follows:

In [ ]:
def damped_oscillator(t, y, gamma, omega0):
    x, v = y
    dydt = [v, -gamma * v - omega0 * x]
    return dydt

So that we can set the values of gamma and omega0 in this function, we can use the [partial](https://docs.python.org/3/library/functools.html#functools.partial) command from [functools](https://docs.python.org/3/library/functools.html). This can be used to generate a version of a function with some parameters fixed at particular values.

Our y0 argument will now have the inital values of x and v. Otherwise we can call `solve_ivp` as before.

In [ ]:
# Then we can use this with SciPy as follows
from functools import partial

dho_sol = scipy.integrate.solve_ivp(fun=partial(damped_oscillator, gamma=0.1, omega0=1),
                                    t_span=(0, 20), y0=[-1, 1], dense_output=True)

plt.plot(tvalues, dho_sol.sol(tvalues)[0], "b-", label="Position")
plt.plot(tvalues, dho_sol.sol(tvalues)[1], "r--", label="Velocity")
plt.legend()
plt.show()

## Methods used for equations of motion

While we saw above we can solve a pair of coupled first order ODEs that are equivalent to a second order ODE. As this is a fairly common problem, such as in molecular dynamics, there are many methods in common use to find the trajectory of the masses in motion. 

One of the most important aspects of a simulation of the equations of motion of a system of interacting particles is that the total energy of the system is conserved. It's also desirable for the methods it be time reversal invarient: if you reverse the sign of the timestep you can get back to your starting point. The methods we've discussed so far can give a small numerical drift in the total energy that can accrue over time, and aren't time reversal invarient. For this reason, methods that will explicitly give time reversal symmetry and conserve the total energy are used instead in these cases.

## Leapfrog Method

The leapfrog method is useful for integrating atomic motion due to its simplicity and stability for oscillatory systems, and as the total energy will be conserved (on average).

Say we have expressions for both acceleration and velocity as before:

$$
v = \frac{\mathrm d x}{\mathrm d t} \\
\frac{\mathrm d v}{\mathrm d t} = F(x).
$$

An Euler integration approach to this would be to calculate a change in atomic position using its velocity for a given timestep. Instead in the leapfrog method, the velocity and position are updated at alternate timesteps. Usually this can be done by starting the velocity half a timestep out of phase, and then continuing as usual. This simple change gives us a second order method instead of a first order method.

We can write down the iterative procedure for timestep $h$ as follows:

$$
x_{n+1} = x_n + v_{n+1/2} h\\
v_{n+3/2} = v_{n+1/2} + a(x_{n+1}) h
$$

To start the process, we also need to find $v_{1/2}$ from $v_0$. This can be done by setting it with the Euler method for a half timestep:  $v_{1/2} = v_0 + \frac{h}{2}a(x_0)$.

Let's write a Python function to do this:

In [ ]:
def leapfrog(accel, x0, v0, dt, nt):
    '''Leapfrog integration to find x(t) and v(t)
    
    Parameters
    ----------
    accel : function
        Function of x that gives the acceleration.
    x0 : float
        Initial value of x.
    v0 : float
        Initial value of v.
    dt : float
        Timestep.
    nt : int
        Number of timesteps
        
    Returns
    -------
    x : float, array
        Position at each timestep.
    '''
    # Initialize an empty arrays that will store x at each timestep.
    x = np.empty(nt)
    # Add in the initial conditions.
    x[0] = x0
    # Put velocity out of sync by half a step (leapfrog method)
    v = v0 + 0.5 * dt * accel(x[0])

    # Use a for loop to iterate over timesteps, updating the velocity
    # and position each time.
    for it in range(1, nt):
        x[it] = x[it-1] + dt * v
        v = v + dt * accel(x[it])
    return x

In [ ]:
# Let's test this for an atom moving in a quartic potential.

# First define a function to return the acceleration.
# a = -(dV/dx) / m
# Let's set the mass to 1 for simplicity.
# Say our potential is -x^2 + 2x^4, then the force
# m=1 is 2x - 8x^3.
def accel_quartic(x):
    return 2*x - 8*x**3

# Now set up our integration:
dt = 0.2
tmax = 30
nt = int(tmax/dt)

# Set up an array storing the t values (for plotting)
tvals = np.linspace(0, tmax, nt)

# Find the trajectory using our leapfrog function for
# two different starting values.
xvals1 = leapfrog(accel_quartic, 0.01, 0.0, dt, nt)
xvals2 = leapfrog(accel_quartic, 0.75, 0.0, dt, nt)

# And plot
plt.plot(tvals, xvals1, label="x0 = 0.01")
plt.plot(tvals, xvals2, label="x0 = 0.75")
plt.xlabel("time")
plt.ylabel("Position")
plt.legend()
plt.show()

### Verlet Integration

The Verlet method is similar in accuracy to the leapfrog method, but has does not explicitly calculate the velocity; instead the position at the current and previous timestep is used to find the next position.

The method can derived by finding the acceleration by taking the central difference approximation to the second derivative of the position at a given step of the simulation:

$$
a_n = \frac{\Delta^2 x_n}{\Delta t^2} =
\frac{\frac{x_{n+1} -x_n}{\Delta t} - \frac{x_n - x_{n-1}}{\Delta t}}{\Delta t} =
\frac{x_{n+1} - 2x_n + x_{n-1}}{\Delta t^2}
$$

Rearranging this we can obtain

$$
x_{n+1} = 2x_n - x_{n-1} + a(x_n)\Delta t^2.
$$

As you might imagine, we need to handle the first timestep a little differently. This can be done by using the initial position and velocity and expanding the position in a Taylor series to second order in terms of $\Delta t$ to find

$$
x_1 = x_0 + v_0 \Delta t + \frac{1}{2}a(x_0) \Delta t^2.
$$

Note this algorithm is also often called the Stormer method.

Let't implement this in Python:

In [ ]:
def verlet(accel, x0, v0, dt, nt):
    '''Verlet integration to find x(t) and v(t)
    
    Parameters
    ----------
    accel : function
        Function of x that gives the acceleration.
    x0 : float
        Initial value of x.
    v0 : float
        Initial value of v.
    dt : float
        Timestep.
    nt : int
        Number of timesteps
        
    Returns
    -------
    x : float, array
        Position at each timestep.
    '''
    # Initialize an empty arrays that will store x at each timestep.
    x = np.empty(nt)
    # Add in the initial conditions.
    x[0] = x0
    # Calculate the x[1] separately
    x[1] = x0 + v0 * dt + 0.5 * accel(x0)*dt**2

    # Use a for loop to iterate over timesteps, updating the position
    # each time.
    for it in range(2, nt):
        x[it] = 2*x[it-1] - x[it-2] + accel(x[it-1])*dt**2
    return x

In [ ]:
# Find the trajectory using our verlet function for
# two different starting values.
xvals3 = verlet(accel_quartic, 0.01, 0.0, dt, nt)
xvals4 = verlet(accel_quartic, 0.75, 0.0, dt, nt)

# And plot
plt.plot(tvals, xvals3, label="x0 = 0.01")
plt.plot(tvals, xvals4, label="x0 = 0.75")
plt.xlabel("time")
plt.ylabel("Position")
plt.legend()
plt.show()

### Velocity Verlet

The velocity Verlet algorithm is more similar to the leapfrog method, execpt that the position and velocity are determined at the same timestep. This can be useful if the velocity of the particles is also needed.

If we take the velocity update from the leapfrog method and consider it to be composed of two half-sized steps we can write down the following sequence of steps:

$$
v_{n+1/2} = v_n + \frac{h}{2}a(x_n)\\
x_{n+1} = x_n + v_{n+1/2} h\\
v_{n+1} = v_{n+1/2} + \frac{h}{2} a(x_{n+1})
$$

This is the velocity Verlet method. It has the advantage of not needing the initial step to be handled separately. You need to be careful not to evaluate $a(x)$ more than once per loop. The value of the current loop can be used in the next loop.

Note: there is also the position Verlet algorithm where the same thing is done, but for position instead of velocity.

Let's look at a Python implementation. You should notice that if you combine the two velocity updates you get back the leapfrog method.

In [ ]:
def velocityverlet(accel, x0, v0, dt, nt):
    '''Velocity Verlet integration to find x(t) and v(t)
    
    Parameters
    ----------
    accel : function
        Function of x that gives the acceleration.
    x0 : float
        Initial value of x.
    v0 : float
        Initial value of v.
    dt : float
        Timestep.
    nt : int
        Number of timesteps
        
    Returns
    -------
    x : float, array
        Position at each timestep.
    '''
    # Initialize an empty arrays that will store x at each timestep.
    x = np.empty(nt)
    # Add in the initial conditions.
    x[0] = x0
    v = v0
    # NB We only want to call this function once per cycle.
    a = accel(x0)
    
    # Use a for loop to iterate over timesteps, updating the velocity
    # and position each time.
    for it in range(1, nt):
        v = v + 0.5*dt*a
        x[it] = x[it-1] + v*dt
        # Sometimes you'l see this with the two preceeding steps
        # combined to one, and a full step update of v
        # at the end
        a = accel(x[it])
        # This is basically equivalent to the leapfrog method
        # except that we have v at that timestep at the end of 
        # each step.
        v = v + 0.5*dt*a
    return x

In [ ]:
# Find the trajectory using our velocity-verlet function for
# two different starting values.
xvals5 = velocityverlet(accel_quartic, 0.01, 0.0, dt, nt)
xvals6 = velocityverlet(accel_quartic, 0.75, 0.0, dt, nt)

# And plot
plt.plot(tvals, xvals3, label="x0 = 0.01")
plt.plot(tvals, xvals4, label="x0 = 0.75")
plt.xlabel("time")
plt.ylabel("Position")
plt.legend()
plt.show()

# Partial Differential Equation Methods

Much of the time, we end up with a set of partial differential equations we need to solve rather than ordinary differential equations, such as from the Schrodinger equation or Maxwell's equation.

The numerical solution of partial differential equations is a very broad topic, with different methods suited to each particular type of problem: such as whether it's an initial value problem or a boundary value problem, or whether the PDEs are parabolic, hyperbolic, elliptic, etc.

If you have a PDE system you need to solve, I suggest examining some more dedicated material to ensure you're using an appropriate numerical method.

I'll cover some of the simpler approaches briefly here.

## Finite-Difference Methods

In these methods finite differences are used to approximate the derivatives. These will often take e.g. a forward difference to approximate the first derivative:

$$
f'(x) \approx \frac{f(x+h)-f(x)}{h}
$$

and e.g. a central difference for the second derivative:

$$
f''(x) \approx \frac{f(x+h)-2f(x)+f(x-h)}{h^2}
$$

Say we're trying to solve some PDE of function in position and time, such as the heat or diffusion equation in one dimension:

$$
\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}.
$$

We could write this as the following $u(t_i, x_j)$:

$$
\frac{u_{i+1, j} - u_{i,j}}{\Delta t} = \frac{u_{i, j+1} - 2u_{i,j}+u_{i,j-1}}{\Delta x^2}
$$

And use this to find an iterative approach:
$$
u_{i+1, j} = (1-2\Delta t/\Delta x^2)u_{i, j} + \Delta t/\Delta x^2 (u_{i, j-1} + u_{i, j+1}).
$$

We would need to start this from some set of initial conditions or boundary values, such as
$u(0,t)=u(1,t)$ and $u(x,0)$ is some set of initial values of x, possibly defined by a function. Then we can obtain all the desired values of $t$ and $x$.

As outlined here, this is an **explicit** method: each step only depends on previously determined values. If we instead used a *backward difference* for the time derivative we could obtain an **implicit** form, in a similar manner as the implict Euler method. This would inclovle solving some linear system at each step, and as with the forward vs backward Euler, would improve the stability of the solution.

### The Crank-Nicolson method

The Crank-Nicolson method is a comboination of the forward and backward methods above, in which we instead use a central difference at time $t_{n+1/2}$ to obtain:

$$
\frac{u_{i+1, j} - u_{i,j}}{\Delta t} = \frac{1}{2}\left(
\frac{u_{i+1, j+1} - 2u_{i+1,j}+u_{i+1,j-1}}{\Delta x^2} +
\frac{u_{i, j+1} - 2u_{i,j}+u_{i,j-1}}{\Delta x^2}
\right).
$$

This gives us a set of linear equations to solve for $u_{i+1, j}$:

$$
(2+2\Delta t/\Delta x^2)u_{i+1,j} - \Delta t/\Delta x^2(u_{i+1,j-1}+u_{i+1,j+1}) =\\
(2+2\Delta t/\Delta x^2)u_{i,j} + \Delta t/\Delta x^2(u_{i,j-1}+u_{i,j+1}).
$$

In this case, this is a tridiagonal system so may be solved relatively efficiently.

## Finite Element

This involves breaking up the region to solve into many small connected parts (finite elements) and is often used for boundary value problems.

Let's illustrate this with an example. Say we want to solve the time-independent Schrodinger equation in 1D on some arbitrary potential.

$$
\left[\frac{-\hbar^2}{2m}\frac{\partial^2}{\partial x^2} + V(x)\right]\psi(x) = E\psi(x)
$$

To simplify the expressions we'll denote $v(x)=2mV(x)/\hbar^2$ and $e=2mE/\hbar^2$, so that the equation we're trying to solve is now

$$
\left[-\frac{\partial^2}{\partial x^2} + v(x)\right]\psi(x) = e\psi(x)
$$

To solve this on our potential we'll assume $\psi$ vanishes outside the input range, and approximate the second derivate of a function sampled at regularly spaced values of $x_i$ in one dimension using the central difference as

$$
f''(x_i) \approx \frac{f(x_{i-1}) - 2f(x_i) + f(x_{i+1})}{\Delta x ^2}
$$

This will lead to an eigenvalue problem for a tridiagonal matrix.

Let's implement this in Python, assuming that we're working in atomic units where $\hbar=1$.

In [ ]:
def fe_tise(V, x0, x1, nx, mass, ):
    '''Finite element solution of the time-independend SE'''
    
    dx = (x1-x0) / nx
    xvals = np.linspace(x0, x1, nx)
    # Evaluate the potential at each x-value
    vvals = 2*mass*V(xvals)
    # Construct the tridiagonal matrix we'll be solving
    # np.full creates an array with a given number of elements, all of the same value
    offdiag = np.full((nx-1), -1/dx**2)
    # The diagonal element has the potential term. v_Ha is an array here, so diag will
    # be an array of the same length.
    diag = 2*mass*vvals + 2/dx**2
    
    # Now call the SciPy tridiagonal eigenvalue function
    eigvals, eigvecs = scipy.linalg.eigh_tridiagonal(diag, offdiag)
    
    #And scale back the eigenvalues
    Evals = eigvals / (2*mass)
    
    return Evals, eigvecs

In [ ]:
# Let's see what we get for a quartic potential

def quartic(x):
    return -x**2 + 2*x**4

eigs, vecs = fe_tise(quartic, x0=-2, x1=2, nx=100, mass=4)

xvals = np.linspace(-2, 2, 100)
# Let's plot the first 3 wavefunctions:
plt.plot(xvals, vecs[:, 0], label="e0=" + str(round(eigs[0], 4)))
plt.plot(xvals, vecs[:, 1], label="e1=" + str(round(eigs[1], 4)))
plt.plot(xvals, vecs[:, 2], label="e2=" + str(round(eigs[2], 4)))
plt.legend()
plt.title("Wavefunctions")
plt.xlabel("Position")
plt.show()

# Try increasing the mass here and see what happens to
# the first two energies and wavefunctions.